In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import PIL.Image as Image
import torch.nn as nn
import torchvision
import torch.optim as optim
import sys
from captum.attr import GuidedGradCam
import cv2
from PIL import Image
from matplotlib.colors import LinearSegmentedColormap
from captum.attr import visualization as viz

sys.path.insert(0, '../src')
from bird_dataset import *
from XAI_birds_dataloader import *
from tqdm import tqdm
from models.multi_task_model import *
from XAI_birds_dataloader import *
from XAI_BirdAttribute_dataloader import *

from download import *

import shutil

In [2]:
# download = True
# id = '156fCp5_VvRfnyHSCBCXhInSE9TxEqDaY'
# destination = '../cub.zip'
# if download:
#     download_file_from_google_drive(id, destination)
# # shutil.unpack_archive('../cub.zip', '../CUB_200_2011')

In [3]:
# #hide
# from fastai.vision.all import *
# from fastai.text.all import *
# from fastai.collab import *
# from fastai.tabular.all import *

In [4]:
bd = BirdDataset(preload=True, attr_file='attributes')

In [5]:
## UNCOMMENT THIS LATER
vgg16 = models.vgg16_bn(pretrained=True)


In [6]:
# vgg16

In [7]:
# bd.images

In [8]:
# [bd.images[i]['attributes'] for i in list(bd.images.keys()) if 'has_bill_shape' in bd.images[i]['attributes']]

In [9]:
# bd.images

In [10]:
trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
train_bird_dataset = Bird_Attribute_Loader(bd, attrs=['has_bill_shape', 'has_breast_pattern'], species=True, transform=trans, train=True)
val_bird_dataset = Bird_Attribute_Loader(bd, attrs=['has_breast_pattern','has_bill_shape'], species=True, transform=trans, train=False, val=True)

In [11]:
# bd.images

In [12]:
# train_bird_dataset.class_dict

In [13]:
train_bird_dataset[1]

{'image': tensor([[[0.2353, 0.1490, 0.1059,  ..., 0.1882, 0.1608, 0.1294],
          [0.2627, 0.1608, 0.0980,  ..., 0.1686, 0.1412, 0.1098],
          [0.3020, 0.1961, 0.1059,  ..., 0.1608, 0.1255, 0.0902],
          ...,
          [0.5373, 0.5020, 0.5333,  ..., 0.4353, 0.4157, 0.4392],
          [0.5882, 0.5373, 0.5412,  ..., 0.4157, 0.4275, 0.4588],
          [0.4980, 0.5686, 0.5843,  ..., 0.4314, 0.4510, 0.4706]],
 
         [[0.3373, 0.2745, 0.2157,  ..., 0.2471, 0.2196, 0.1922],
          [0.3451, 0.2667, 0.1882,  ..., 0.2392, 0.2118, 0.1804],
          [0.3647, 0.2784, 0.1843,  ..., 0.2353, 0.2000, 0.1647],
          ...,
          [0.5216, 0.5098, 0.5765,  ..., 0.3647, 0.3216, 0.3412],
          [0.5373, 0.5333, 0.5490,  ..., 0.3333, 0.3216, 0.3451],
          [0.4549, 0.5529, 0.6039,  ..., 0.3451, 0.3412, 0.3529]],
 
         [[0.2941, 0.2275, 0.1843,  ..., 0.2510, 0.2314, 0.2000],
          [0.3137, 0.2314, 0.1725,  ..., 0.2431, 0.2196, 0.1882],
          [0.3451, 0.2549, 0.17

In [14]:
# train_bird_dataset[0]

In [15]:
model = MultiTaskModel(vgg16, train_bird_dataset)

In [16]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# If 'cuda:0' is printed, it means GPU is available.

if torch.cuda.is_available():
    model.cuda()

batch_size = 1
trainloader = DataLoader(train_bird_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_bird_dataset, batch_size=batch_size, shuffle=True)

loss_func = MultiTaskLossWrapper().to(device)
opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

cpu


In [17]:
data = train_bird_dataset[0]
inputs, labels = data['image'], torch.LongTensor(data['labels'])

In [18]:
inputs.size()

torch.Size([3, 224, 224])

In [19]:
labels

tensor([ 6,  0, 16])

In [20]:
loss_func(inputs, labels)

tensor(32.6740)

In [21]:
%%time
avg_losses = []
avg_val_losses = []
epochs = 50
print_freq = 100
val_acc = []
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs.
#         print("LABELS:",data['labels'])
        inputs, labels = data['image'], torch.LongTensor(data['labels'])

        # Move the inputs to the specified device.
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients.
        opt.zero_grad()

        # Forward step.
        outputs = model(inputs)
        # print('outputs came')
        # print("OUTPUTS:",outputs) 
        # break
        loss = loss_func(outputs, labels)
        # print(loss)
        # Backward step.
        loss.backward()
        
        # Optimization step (update the parameters).
        opt.step()

        # Print statistics.
        running_loss += loss.item()
#         print('running loss', running_loss)
#         print(outputs)
        if i % print_freq == print_freq - 1: # Print every several mini-batches.
            avg_loss = running_loss / print_freq
            print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}'.format(
                epoch, i, avg_loss))
            running_loss = 0.0
            
    avg_losses.append(avg_loss)
    
    model.eval()
    with torch.no_grad():
        num_correct=0
        val_losses = []
        data_iter = iter(valloader)
        for val_data in data_iter:
            val_inputs, val_labels = val_data['image'].cuda(), torch.LongTensor(val_data['labels']).cuda()
#                     print('val inputs: ',val_inputs.size())
#                     print('val labels: ',val_labels)
#                     print('inputs: ',inputs.size())
#             print(val_labels)
            val_outputs = model(val_inputs, train_bird_dataset)
#                     print('val outputs: ',val_outputs)
#                     print('val loss: ',nn.CrossEntropyLoss()(val_outputs, val_labels))
            opt.zero_grad() #zero the parameter gradients
            _, val_predicted = [torch.max(i, 1) for i in val_outputs]
            #     print(predicted)
            # print(labels)
            # print(predicted)
#             num_correct_k += topk_accuracy(k, labels, outputs)
            num_correct += sum(np.array(val_labels.cpu())==np.array(val_predicted))
#             print(predicted)
#             print(val_labels)
            val_losses.append(loss_func(val_outputs, val_labels).item())
        acc = num_correct/(len(data_iter)*batch_size)
        val_acc.append(acc)
        print('Validation accuracy:',acc)
        print('Average validation loss:',np.mean(val_losses))
        avg_val_losses.append(np.mean(val_losses))
    model.train()
print('Finished Training.')

tensor(8.0168, grad_fn=<AddBackward0>)
tensor(7.6753, grad_fn=<AddBackward0>)
tensor(8.8603, grad_fn=<AddBackward0>)
tensor(9.3084, grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [25]:
torch.save(model.state_dict(), '../models/transfer_multi-task_2_50_epoch_state_dict.pth')

In [26]:
(avg_losses)

[0.52,
 0.5,
 0.54,
 0.55,
 0.55,
 0.51,
 0.5,
 0.56,
 0.45,
 0.53,
 0.52,
 0.55,
 0.62,
 0.58,
 0.59,
 0.58,
 0.58,
 0.56,
 0.52,
 0.6,
 0.54,
 0.56,
 0.34,
 0.47,
 0.54,
 0.62,
 0.62,
 0.53,
 0.65,
 0.61,
 0.54,
 0.55,
 0.66,
 0.71,
 0.68,
 0.65,
 0.63,
 0.64,
 0.65,
 0.7,
 0.69,
 0.65,
 0.66,
 0.66,
 0.55,
 0.68,
 0.69,
 0.67,
 0.65,
 0.65]

In [40]:
labels[0]

tensor(7, device='cuda:0')

In [44]:
model(inputs)

(tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
        device='cuda:0', grad_fn=<AddmmBackward>))

In [45]:
model

MultiTaskModel(
  (encoder): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, paddin

In [42]:
outputs

(tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
        device='cuda:0', grad_fn=<AddmmBackward>))

In [41]:
outputs[0].reshape(1, -1)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AsStridedBackward>)

In [125]:
labels[0]

tensor(6., device='cuda:0')

In [128]:
outputs[0].size()

torch.Size([1, 9])

In [131]:
labels[0]

tensor(11., device='cuda:0')

In [129]:
outputs[1].size()

torch.Size([1, 15])

In [126]:
nn.CrossEntropyLoss()(outputs, labels)

AttributeError: 'tuple' object has no attribute 'log_softmax'